# HW5
## Part I: Statistical significance of regression

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.ma as ma

In [2]:
# time range 1884-2020
tr=[1884,2020]
df=pd.read_excel('LAmeantempdata.xlsx')
data=df.to_numpy()
data[data==-999]=np.nan
x0=data[:,0]
tind=(x0>=tr[0]) & (x0<=tr[1])
year=data[tind,0] # time
apr1=data[tind,4] # April
nov1=data[tind,11] # November

In [3]:
# calculate 95% CI of April temperature trend
# covariance for April
gd=(~np.isnan(apr1)) # check if missing data
N=np.size(year)
df=N-2
c=np.cov(year[gd],apr1[gd])
a1=c[0,1]/c[0,0]
b1=np.mean(apr1[gd])-a1*np.mean(year[gd])
MSE=np.sum((apr1[gd]-(a1*year[gd]+b1))**2)/(N-2)
SE2=MSE/np.sum((year[gd]-np.mean(year[gd]))**2)
SE=np.sqrt(SE2)
CI=2*SE
print('SE of the linear trend is '+str(SE.round(3)))
print('Linear trend of April is '+str(a1.round(3))+'\nconfidence interval (+/-) '+str(CI.round(3))+' degF/yr')

SE of the linear trend is 0.005
Linear trend of April is 0.044
confidence interval (+/-) 0.01 degF/yr


In [4]:
# calculate 95% CI of November temperature trend
# covariance for November
gd=(~np.isnan(nov1)) # check if missing data
N=np.size(year)
df=N-2
c=np.cov(year[gd],apr1[gd])
a1=c[0,1]/c[0,0]
b1=np.mean(nov1[gd])-a1*np.mean(year[gd])
MSE=np.sum((nov1[gd]-(a1*year[gd]+b1))**2)/(N-2)
SE2=MSE/np.sum((year[gd]-np.mean(year[gd]))**2)
SE=np.sqrt(SE2)
CI=2*SE
print('SE of the linear trend is '+str(SE.round(3)))
print('Linear trend of November is '+str(a1.round(3))+'\nconfidence interval (+/-) '+str(CI.round(3))+' degF/yr')

SE of the linear trend is 0.006
Linear trend of November is 0.044
confidence interval (+/-) 0.012 degF/yr


## Part II: Testing statistical significance of correlation

In [5]:
# time range 1947-2020 becuase NOLA data does not 
#tr=[1947,2020]
df1=pd.read_excel('LANOLAmeantempdata.xlsx',sheet_name='LA')
df2=pd.read_excel('LANOLAmeantempdata.xlsx',sheet_name='NOLA')

# pick a month
month='January' # January
year=df1['Year']
N=np.size(year)
Temp=np.zeros((N,2)) # prepare an empty array

In [6]:
# 2 cities
city=['LA', 'NOLA']
Temp[:,0]=df1[month]
Temp[:,1]=df2[month]
# create the "Mask" for missing data
Mask=(Temp==-999)
Tempma=ma.array(Temp,mask=Mask)

In [7]:
tr=[1900,2020]
gd=(year>=tr[0]) & (year<=tr[1])
D=Tempma[gd,:]
c=np.ma.corrcoef(D.T)
df=pd.DataFrame(c,columns=city)
df

,LA,NOLA
0,1.00000,-0.35574
1,-0.35574,1.00000


In [8]:
# Let's perform significance test with 95% confidence level (2 tail)
tval = np.abs(c)*np.sqrt((N-2)/(1-c**2))
# since N > 30, we can use Gaussian distribution
tcrit= 2.0
test = (tval) > tcrit
df=pd.DataFrame(test,columns=city)
df

,LA,NOLA
0,NaN,True
1,True,NaN


In [9]:
#detrend data
from scipy import signal
Temp[:,0]=signal.detrend(df1[month])
Temp[:,1]=signal.detrend(df2[month])

In [10]:
tr=[1900,2020]
gd=(year>=tr[0]) & (year<=tr[1])
D=Tempma[gd,:]
c=np.ma.corrcoef(D.T)
df=pd.DataFrame(c,columns=city)
df

,LA,NOLA
0,1.000000,-0.409903
1,-0.409903,1.000000


In [11]:
# Let's perform significance test with 95% confidence level (2 tail)
tval = np.abs(c)*np.sqrt((N-2)/(1-c**2))
# since N > 30, we can use Gaussian distribution
tcrit= 2.0
test = (tval) > tcrit
df=pd.DataFrame(test,columns=city)
df

,LA,NOLA
0,NaN,True
1,True,NaN


### Yes, the correlation coffeficient changed when the data was detrended. The cities are negatively correlated. 